In [20]:
# Standard packages
from skimage import morphology

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import SimpleITK as sitk
import porespy as ps
from radiomics import featureextractor, shape, firstorder, glcm, glrlm

from collections import Counter

from multiprocessing import Pool

# Custom packages
import auxiliary.values as v
from filtering import cardiac_region as cr
from auxiliary.data.dataset_ht import HtDataset
from auxiliary.utils.colors import bcolors as c
from auxiliary.data import imaging

In [35]:
specimen = '0806_E6'

ds = HtDataset()
path_nu = ds.read_features(specimen, 'Nuclei', 'myocardium')
path_mem = ds.read_features(specimen, 'Membrane', 'myocardium')

nuclei = pd.read_csv(path_nu)
membrane = pd.read_csv(path_mem)

print(len(nuclei), len(membrane))

2578 1656


Count number of nuclei inside each membrane

In [36]:
feat_nu = pd.DataFrame(
    columns=['x', 'y', 'z', 'volume'],
    data=[
        [int(x), int(y), int(z), vol] 
        for (x, y, z), vol 
        in zip(
            list(map(
                lambda aux: aux.replace('[', '').replace(']', '').replace(',', ' ').split(), 
                nuclei.centroids
            )),
            nuclei.volumes
        )
    ]
)

feat_mem = pd.DataFrame(
    columns=['x', 'y', 'z', 'volume'],
    data=[
        [int(x), int(y), int(z), vol] 
        for (x, y, z), vol 
        in zip(
            list(map(
                lambda aux: aux.replace('[', '').replace(']', '').replace(',', ' ').split(), 
                membrane.centroids
            )),
            membrane.volumes
        )
    ]
)

In [37]:
fig = go.Figure(data=[
    go.Scatter3d(
        x=feat_nu['x'],
        y=feat_nu['y'],
        z=feat_nu['z'],
        mode='markers',
        marker=dict(
            size=3,
            color='blue',
            # colorscale='vidris',
            opacity=0.75
        )
    ), 
    go.Scatter3d(
        x=feat_mem['x'],
        y=feat_mem['y'],
        z=feat_mem['z'],
        mode='markers',
        marker=dict(
            size=3,
            color='green',
#             colorscale='vidris',
            opacity=0.75
        )
    )],
    layout=go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
    )
)

fig.show()

In [38]:
for i, (x, y, z, vol) in enumerate(feat_mem.values):
    if i > 5:
        break
    print(f'Centroid: {x}, {y}, {z}')
    print(f'Volume: {vol}')
    
    for j, (x_n, y_n, z_n, vol_n) in enumerate(feat_nu.values):
        if (x_n > x - vol) & (x_n < x + vol) & (y_n > y - vol) & (y_n < y + vol) & (z_n > z - vol) & (z_n < z + vol):
            print(f'Nucleus inside: {x_n}, {y_n}, {z_n}')

Centroid: 74.0, 473.0, 202.0
Volume: 4108.0
Nucleus inside: 80.0, 467.0, 169.0
Nucleus inside: 81.0, 494.0, 184.0
Nucleus inside: 86.0, 493.0, 217.0
Nucleus inside: 87.0, 468.0, 206.0
Nucleus inside: 85.0, 495.0, 210.0
Nucleus inside: 85.0, 495.0, 209.0
Nucleus inside: 88.0, 452.0, 194.0
Nucleus inside: 90.0, 460.0, 175.0
Nucleus inside: 96.0, 454.0, 156.0
Nucleus inside: 93.0, 467.0, 193.0
Nucleus inside: 94.0, 467.0, 188.0
Nucleus inside: 94.0, 480.0, 216.0
Nucleus inside: 90.0, 508.0, 209.0
Nucleus inside: 98.0, 446.0, 178.0
Nucleus inside: 96.0, 475.0, 201.0
Nucleus inside: 93.0, 500.0, 206.0
Nucleus inside: 96.0, 504.0, 213.0
Nucleus inside: 94.0, 524.0, 192.0
Nucleus inside: 96.0, 435.0, 166.0
Nucleus inside: 99.0, 435.0, 171.0
Nucleus inside: 98.0, 438.0, 160.0
Nucleus inside: 96.0, 442.0, 190.0
Nucleus inside: 96.0, 480.0, 175.0
Nucleus inside: 96.0, 476.0, 195.0
Nucleus inside: 98.0, 439.0, 155.0
Nucleus inside: 100.0, 454.0, 195.0
Nucleus inside: 98.0, 467.0, 184.0
Nucleus in